In [1]:
import pandas as pd

In [7]:
enru = pd.read_csv("./lookup results/yandex_lookup_en_ru_lemmas_only_WHOLE_LIST.csv")

In [8]:
enru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 5 columns):
Unnamed: 0        1410 non-null int64
word              1410 non-null object
local_word        1410 non-null object
examples          1410 non-null object
local_examples    1410 non-null object
dtypes: int64(1), object(4)
memory usage: 55.2+ KB


In [11]:
enru.head(10)

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,для,"['for the bank', 'for last month', 'for one ye...","['для банка', 'за последний месяц', 'на один г..."
1,1,for,ради,['for god'],['ради бога']
2,2,for,в течение,[],[]
3,3,for,в отношении,[],[]
4,4,for,для целей,[],[]
5,5,for,ибо,[],[]
6,6,for,чтобы,[],[]
7,7,before,до,['before the war'],['до войны']
8,8,before,впереди,[],[]
9,9,before,пред,[],[]


In [13]:
enrufr_naive = pd.read_csv("./lookup results/naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")
enrufr_naive.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe..."
1,1,for,pour,['for god'],['pour dieu']
2,2,for,au cours de,[],[]
3,3,for,à l'égard de,[],[]
4,4,for,à des fins,[],[]


In [16]:
enrufr_naive['russian_word'] = list(enru['local_word'])

In [19]:
del (enrufr_naive['Unnamed: 0'])
enrufr_naive.head()

,word,local_word,examples,local_examples,russian_word
0,for,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe...",для
1,for,pour,['for god'],['pour dieu'],ради
2,for,au cours de,[],[],в течение
3,for,à l'égard de,[],[],в отношении
4,for,à des fins,[],[],для целей


In [21]:
column = ['word','russian_word', 'local_word','examples','local_examples']
enrufr_naive = enrufr_naive.reindex(columns=column)

In [26]:
enrufr_naive.head(60)

,word,russian_word,local_word,examples,local_examples
0,for,для,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe..."
1,for,ради,pour,['for god'],['pour dieu']
2,for,в течение,au cours de,[],[]
3,for,в отношении,à l'égard de,[],[]
4,for,для целей,à des fins,[],[]
5,for,ибо,car,[],[]
6,for,чтобы,pour,[],[]
7,before,до,avant,['before the war'],['avant la guerre']
8,before,впереди,en avant de,[],[]
9,before,пред,devant,[],[]


In [24]:
enrufr_naive.to_csv("enrufr_naive_transation_yandex_lookup_whole_list.csv")

# match systran eng fr

In [28]:
systran_en_fr = pd.read_csv("systran_en_fr.csv")
systran_en_fr.head()

,Unnamed: 0,word,local_word,examples,local_examples,definitions
0,0,word,mot,key word|last word|signal word|first words|,mot clé | dernier mot | mot indicateur | premi...,term | remark |
1,1,word,parole,fine words|word of god|kind word|word of the l...,belle parole | parole de dieu | aimables parol...,floor | speech | lyric | hon | voice |
2,2,word,terme,following words|search word|operative word|eng...,termes suivants | terme de recherche | terme p...,term | end | stop |
3,3,word,propos,written word|wise words|clear words|warm words|,propos écrits ici | propos sages | propos clai...,comment | remark | hon | rhetoric | pertinence...
4,4,word,mention,words of limitation|,mention de limitation |,particular | reference | mention | statement |...


In [29]:
systran_en_ru = pd.read_csv("systran_en_ru.csv")
systran_en_ru.head()

,Unnamed: 0,word,local_word,examples,local_examples,definitions
0,0,word,слово,key word|english word|first word|good word|,ключевое слово | английское слово | первое сло...,speech | expression | term |
1,1,and,и,NaN,NaN,|
2,2,and,а,NaN,NaN,but | while |
3,3,and,и,He thinks of you day and night|,Он думает о вас день и ночь |,NaN
4,4,and,но,It's an embarrassing joke and not an insult|,"Это неуместная шутка, а не оскорбление |",NaN


In [62]:
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_distances
from  sklearn.metrics.pairwise import cosine_similarity

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

import numpy as np

In [31]:
model = api.load('glove-twitter-100')

In [47]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [65]:
def get_nonstopwords_vectors_sum(lemm_list):
    orig_vector = np.zeros(shape = 100)
    for lemma in lemm_list:
        if lemma not in stopWords:
            print (lemma)
            orig_vector += model[lemma]
    return orig_vector


In [53]:
def get_lemmatized_line(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return lemmatized_line_list

get_lemmatized_line(l1)

['hello', 'my', 'friend']

In [ ]:
def sum_up_vectors

In [67]:
l1 = "hello my friends"
l2 = "hello my girlfriend"

def find_lines_similarity(line_1, line_2):
    lemmatized_l1_list = get_lemmatized_line(line_1)
    lemmatized_l2_list = get_lemmatized_line(line_2)
    print(lemmatized_l1_list,lemmatized_l2_list )
    l1_vect = get_nonstopwords_vectors_sum(lemmatized_l1_list)
    l2_vect = get_nonstopwords_vectors_sum(lemmatized_l2_list)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    print(cos_sim)

find_lines_similarity(l1, l2)    
    

['hello', 'my', 'friend'] ['hello', 'my', 'girlfriend']
hello
friend
hello
girlfriend
0.9236770700727499


In [41]:
cosine_similarity(model['mother'].reshape(1, -1), model['woman'].reshape(1, -1))[0][0]

0.7815035